### Prerequisites

#### Sentinel Hub account

In order to use Sentinel Hub services you will need a Sentinel Hub account. If you do not have one yet, create one at [Sentinel Hub webpage](https://services.sentinel-hub.com/oauth/subscription).

Once you have the account set up, login to [Sentinel Hub Configurator](https://apps.sentinel-hub.com/configurator/). Inside there will already exist one configuration with an **instance ID** (alpha-numeric code of length 36). For this tutorial it is recommended that you create a new configuration (`"Add new configuration"`) and set the configuration to be based on **Python scripts template**. Such configuration will already contain all layers used in these examples. Otherwise you will have to define the layers for your  configuration yourself.

After you have decided which configuration to use, you have two options You can either put configuration's **instance ID** into `sentinelhub` package's configuration file following the [configuration instructions](http://sentinelhub-py.readthedocs.io/en/latest/configure.html) or you can write it down in the following cell:

In [ ]:
INSTANCE_ID = 'YOUR-INSTANCE-ID-FROM-HUB' # get an API key from sentinel hub

#### Imports 

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import random
import time

In [ ]:
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, CustomUrlParam, DataSource

Plot images in output.

In [ ]:
def plot_image(image, factor=1):
    """
    Utility function for plotting RGB images.
    """
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))
    
    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)

Method to crop images to a size of 2000x2000 pixels.<br>
**This was needed for my project.** Change it the way you like :)

In [ ]:
def crop_img(img):
    startx = 0
    starty = 0
    endx = 2000
    endy = 2000
    return img[starty:endy,startx:endx]

Analyse image and return true iff the images does not contain transparent pixels.<br>
**This was needed for my prject.** Change it the way you like.

In [ ]:
def analyse_img(img):
    return np.all(np.any(img, axis=2)) == True

Method to calculate the lower right and upper left corner of the **Betsiboka Bounding Box** used to request the satellite image

In [ ]:
def GetBoundingCoords(centerLong, centerLat):
    distance = 75 # km
    centerLong = (centerLong + 540) % 360 - 180 # normalise the center longitude
    
    right = MaxLatLongOnBearing(centerLong, centerLat, 135, distance)
    left = MaxLatLongOnBearing(centerLong, centerLat, 315, distance)
    
    return [right, left]

Method to calculate the lat and long of point a given a certain direction and a distance to it.

`d = distance`<br>
`d/6371 = distance divided by the radius of the earth`<br>
`bearing = clockwise from north`

In [ ]:
def MaxLatLongOnBearing(centerLong, centerLat, bearing, d):
    lon1 = math.radians(centerLong) # translate degrees to radians
    lat1 = math.radians(centerLat) # translate degrees to radians
    
    brng = math.radians(bearing) # translate degrees to radians
    
    lat2 =math.asin(math.sin(lat1) * math.cos(d / 6371) + math.cos(lat1) * math.sin(d / 6371) * math.cos(brng))
    lon2 =lon1 + math.atan2(math.cos(d / 6371) - math.sin(lat1) * math.sin(lat2), math.sin(brng) * math.sin(d / 6371) * math.cos(lat1))
    
    maxLat = math.degrees(lat2)
    maxLong = math.degrees(lon2)
    if centerLong < 0:
        maxLong -= 90
        
    maxLong = (maxLong + 540) % 360 - 180 # normalise longitude

    return [round(maxLong, 3), round(maxLat, 3)]

Set the variables for the retrieval

In [ ]:
image_counter = 0
counter = 1
random_counter = 10001
images = [] # list of images

Retrieve images with API of sentinel hub.<br>
**This can take some time.** _For my purposes it took 4 days to get enough images to train a classifier..._<br>
If you get an error message, try over a few minutes to run the query again.

In [ ]:
random.seed(random_counter) # to reproduce results
print(f'{"Counter":7} {"Time":8} {"Random Seed":11} {"Image Counter":13} {"Box":10}')
while counter <= 3000:
    lat = random.uniform(-80, 84) # random latitude between -80 and 84 degrees
    long = random.uniform(-180, 180) # random longitude between -180 and 180 degrees
    box=GetBoundingCoords(long, lat) # create the bounding box with center point in lat and long
    
    BBOX = BBox(bbox=box, crs=CRS.WGS84) # translate thebox to a system sentinel understands
    
    wms_img_request = WcsRequest(layer='CLOUDS-90', # please change/create layer in sentinel hub profile page
                                 bbox=BBOX,
                                 resx='50m', resy='50m', # one pixel equal 50x50m resolution
                                 # maxcc=1.0, # max cloud coverage
                                 image_format=MimeType.PNG, # image type
                                 instance_id=INSTANCE_ID, # your hub ID
                                 # time_difference=datetime.timedelta(days=10), # this is used to merge pictures across several hours/days
                                 custom_url_params={CustomUrlParam.TRANSPARENT: True}) # set background to transparent
    
    img = wms_img_request.get_data() # request the data
    st = datetime.datetime.fromtimestamp(time.time()).strftime('%H_%M_%S') # time stamp for progress and filename
    if len(img) > 0: # if image was retrieved
        img = crop_img(img[-1]) # select the last one taken
        images.append(img) # append images to list
        if analyse_img(img): # analyse images for transparent pixels
            print(f'{counter:7} {st:8} {random_counter:11} {image_counter:13} ' + str(box))
            # image name contains the image counter (how many requests have been done),
            # the random counter (to repeat the results),
            # the time stamp,
            # and the coordinates of the bounding box
            # change it the way you like
            image_name = str(image_counter) + "_" + str(random_counter) + "_" + st + "_" + str(box[0][0]) + "_" + str(box[0][1]) + "_" + str(box[1][0]) + "_" + str(box[1][1]) + ".png"
            mimg.imsave("./images_sentinel/" + image_name, img)
            counter += 1
            random_counter += 1
            random.seed(random_counter)
        else:
            print(f'{"----":7} {st:8} {"----":11} {image_counter:13} ' + str(box)) # image with transparent pixels found
    else:
        print(f'{"****":7} {st:8} {"****":11} {image_counter:13} ' + str(box)) # no image found
    image_counter += 1